# Portefólio - Grupo 3



*   André Dias Ferreira - PG55127
*   João Manuel Barbosa Lima - PG55701
*   Romeu Fernandes - PG45861
*  Inês Lameira - PG40082
*   André Gomes - PG





# Sequências

#Descrição do algoritmo

O algoritmo realiza a análise de sequências de DNA para identificar ORFs (Open Reading Frames) e proteínas, funcionando em quatro etapas principais:



1.   **Extração dos Codões**: O algoritmo extrai codões (sequências de 3 nucleotídeos) de uma sequência de DNA a partir de diferentes quadros de leitura (0, 1, 2).

2.   **Tradução para Aminoácidos**: Cada codão é traduzido no seu aminoácido correspondente, utilizando uma tabela de codões, iniciando com um star codon e terminando com um stop codon. O algoritmo também permite a utilização de múltiplos codões de início.

3. **Identificação de Proteínas**: O algoritmo analisa a sequência de aminoácidos para identificar proteínas completas, ou seja, aquelas que iniciam com um codão de início (geralmente "M" para metionina) e terminam com um stop codon.

4. **Identificação de ORFs e Proteínas**: A partir das traduções geradas, o algoritmo encontra todos os ORFs possíveis e as proteínas associadas, considerando todos os quadros de leitura retornando e imprimindo esses resultados.




# Projeto de alto nível

#Input:


*   **Sequência de DNA:** A sequência de nucleotídeos (A, T, G, C) que será analisada para a identificação de ORFs e proteínas.
*   **Start Codons (Opcional):** Uma lista de codões de iniciação, com o valor padrão sendo **ATG, CTG, TTG**, que define quais sequências serão consideradas como início para a tradução.
* **Stop Codons** (Opcional): Uma lista de codões de terminação, com o valor padrão sendo **TAA, TAG, TGA**, que indicam onde as proteínas terminam. Se não forem fornecidos, o algoritmo usará os codões de parada padrão.

#Processo:

1. **Identificação de Codões:** O algoritmo percorre a sequência de DNA e extrai os codões, que são substrings de três nucleotídeos. Essa extração ocorre em três quadros de leitura diferentes, iniciando com os índices 0, 1 e 2 da sequência.

2. **Verificação de Codões de Início:** O algoritmo verifica os codões de iniciação definidos e começa a tradução a partir do primeiro start codon encontrado. Para cada quadro de leitura, a sequência de codões é analisada em busca de um codão de iniciação.

3. **Tradução para Aminoácidos:** Após identificar um codão de iniciação, o algoritmo traduz os codões subsequentes para seus aminoácidos correspondentes, com base na tabela de codões. A tradução continua até que um codão de terminação seja encontrado.

4. **Identificação de ORFs (Open Reading Frames):** O algoritmo encontra todos os ORFs possíveis nas três leituras (frames) da sequência. Um ORF é uma sequência de DNA entre um start codon e um stop codon, que pode ser traduzida numa proteína. O algoritmo identifica todos os ORFs válidos em cada quadro de leitura.

5. **Tradução de Proteínas:** Para cada ORF identificado, o algoritmo gera a sequência de aminoácidos correspondente, e as proteínas completas são armazenadas. As proteínas são aquelas que começam com metionina (codão "ATG") e terminam com um codão de terminação.

6. **Retorno dos Resultados:** O algoritmo retorna todos os ORFs encontrados e as proteínas correspondentes para cada quadro de leitura.



# Projeto de baixo nível

# Implementação


In [ ]:
def rf_codons(strand, reading_frame=0):
    assert isinstance(strand, str)
    assert 0 <= reading_frame <= 2 and isinstance(reading_frame, int)
    return [strand[i:i+3].upper() for i in range(reading_frame, len(strand)-2, 3)]

def all_rfs(strand):
    assert isinstance(strand, str)
    return [rf_codons(strand, i) for i in range(3)]

def codon_table():
    return {
        "ATA":"I", "ATC":"I", "ATT":"I", "ATG":"M", "ACA":"T", "ACC":"T", "ACG":"T", "ACT":"T",
        "AAC":"N", "AAT":"N", "AAA":"K", "AAG":"K", "AGC":"S", "AGT":"S", "AGA":"R", "AGG":"R",
        "CTA":"L", "CTC":"L", "CTG":"L", "CTT":"L", "CCA":"P", "CCC":"P", "CCG":"P", "CCT":"P",
        "CAC":"H", "CAT":"H", "CAA":"Q", "CAG":"Q", "CGA":"R", "CGC":"R", "CGG":"R", "CGT":"R",
        "GTA":"V", "GTC":"V", "GTG":"V", "GTT":"V", "GCA":"A", "GCC":"A", "GCG":"A", "GCT":"A",
        "GAC":"D", "GAT":"D", "GAA":"E", "GAG":"E", "GGA":"G", "GGC":"G", "GGG":"G", "GGT":"G",
        "TCA":"S", "TCC":"S", "TCG":"S", "TCT":"S", "TTC":"F", "TTT":"F", "TTA":"L", "TTG":"L",
        "TAC":"Y", "TAT":"Y", "TAA":"_", "TAG":"_", "TGC":"C", "TGT":"C", "TGA":"_", "TGG":"W"
    }

def start_codons():
    return ['ATG', 'CTG', 'TTG']

def stop_codons():
    return ['TAA', 'TAG', 'TGA']

def translate_rf(all_rf_codons, start_codon=None):
    trans_table = codon_table()
    start_codon = start_codon or start_codons()

    assert isinstance(start_codon, list) and all(isinstance(codon, str) for codon in start_codon)

    seq_aas = []
    for rf in all_rf_codons:
        seq_aa = ''
        in_orf = False
        for codon in rf:
            upper_codon = codon.upper()
            if upper_codon in stop_codons() and in_orf:
                seq_aa += '_'  # Adiciona codão stop como '_'
                break
            elif upper_codon in start_codon and not in_orf:
                in_orf = True
                seq_aa += trans_table[upper_codon]  # Início da tradução com o aminoácido correspondente
            elif in_orf:
                seq_aa += trans_table[upper_codon]  # Adiciona o aminoácido correspondente
        seq_aas.append(seq_aa)
    return seq_aas

def find_prots(seq_aa):
    assert isinstance(seq_aa, str)
    alphabet = 'ACDEFGHIKLMNPQRSTVWY_'
    prots = []
    current_prot = ''
    in_protein = False

    for aa in seq_aa.upper():
        assert aa in alphabet
        if in_protein:
            if aa == '_':  # Codão stop encontrado
                in_protein = False
                if current_prot:  # Proteína não vazia
                    prots.append(current_prot)
                current_prot = ''
            else:
                current_prot += aa
        elif aa not in '_':  # Início da proteína, qualquer aminoácido exceto '_'
            current_prot = aa
            in_protein = True
    # Adicionar proteína se não tiver terminado em '_'
    if current_prot:
        prots.append(current_prot)
    return prots

def all_prots(strand, start='', end=''):
    rfs_cod = all_rfs(strand)
    rfs_seq_aa = translate_rf(rfs_cod, start_codon=[start])
    todas_proteinas = set()
    for seq in rfs_seq_aa:
        prots = find_prots(seq)
        todas_proteinas.update(prots)

    orfs = []
    for i, seq in enumerate(rfs_seq_aa):
        for prot in find_prots(seq):
            start_idx = seq.find(prot)
            end_idx = start_idx + len(prot)
            # Ajuste para garantir que estamos a usar os codões corretos
            # Dividir por 3 porque cada aminoácido corresponde a 3 nucleotídeos
            orfs.append(''.join(rfs_cod[i][start_idx//3:end_idx//3]))  # Ajuste para garantir que o ORF está correto

    return orfs, todas_proteinas

In [ ]:
strand=input("Digite a sequência de DNA: ").upper()
#Exemplo do funcionamento da função all_prots variando o codão de iniciação e terminação
#print('Exemplo variando o codão de iniciação:')
#print('Com ATG:', all_prots(strand,start='ATG',end=''))
#print('Com CTG:', all_prots(strand,start='CTG'))
#print('Com TTG:', all_prots(strand,start='TTG'))
print(all_prots(strand))

# Testes Unitários

In [ ]:
import unittest

class Test_ORFs(unittest.TestCase):

    def test_rf_codons(self):
        strand = "ATGCGATATAGG"
        expected = ["ATG", "CGA", "TAT", "AGG"]
        self.assertEqual(rf_codons(strand), expected)
        self.assertEqual(rf_codons(strand, reading_frame=1), ["TGC", "GAT", "ATA"])

        # Teste de erro
        with self.assertRaises(AssertionError):
            rf_codons(123)  # strand não é uma string
        with self.assertRaises(AssertionError):
            rf_codons(strand, reading_frame=3)  # reading_frame inválido

    def test_all_rfs(self):
        strand = "ATGCGATATAGG"
        result = all_rfs(strand)
        self.assertEqual(len(result), 3)  # Deve haver 3 reading frames
        self.assertEqual(result[0], ['ATG', 'CGA', 'TAT', 'AGG'])
        self.assertEqual(result[1], ['TGC', 'GAT', 'ATA'])
        self.assertEqual(result[2], ['GCG', 'ATA', 'TAG'])


        with self.assertRaises(AssertionError):
            all_rfs(123)  # strand não é uma string

    def test_codon_table(self):
        table = codon_table()
        self.assertEqual(table['ATG'], 'M')
        self.assertEqual(table['TAA'], '_')

    def test_start_stop_codons(self):
        self.assertEqual(start_codons(), ['ATG', 'CTG', 'TTG'])
        self.assertEqual(stop_codons(), ['TAA', 'TAG', 'TGA'])

    def test_translate_rf(self):
        strand = "ATGTAATAG"
        rfs = all_rfs(strand)
        expected = ['M_', '', '']
        self.assertEqual(translate_rf(rfs), expected)

        # Teste com múltiplos start cod
        strand = "ATGGTGTAA"
        rfs = all_rfs(strand)
        expected = ['MV_', '', '']
        self.assertEqual(translate_rf(rfs, start_codon=['ATG', 'CTG', 'TTG']), expected)

        # Teste de erro
        with self.assertRaises(AssertionError):
            translate_rf(rfs, start_codon='XXX')  # cod start inválido

    def test_find_prots(self):
        seq = "MAAAAAAAAAAAAAAA_"
        self.assertEqual(find_prots(seq), ['MAAAAAAAAAAAAAAA'])

        # Teste de erro
        with self.assertRaises(AssertionError):
            find_prots('MZ')  # Aminoácido inválido

def test_all_prots(self):
    strand = "ATGTAA"
    orfs, prots = all_prots(strand)
    self.assertEqual(orfs, ['ATG'])
    self.assertEqual(prots, ['M'])

    # Teste com uma sequência mais complexa, incluindo múltiplos codões de início
    strand = "ATGATATAGCTGTAA"
    orfs, prots = all_prots(strand, start='ATG')  # Teste com um início específico
    self.assertIn('ATG', orfs)  # Espera-se o ORF 'ATG'
    self.assertIn('M', prots)  # Espera-se a proteína 'M'

    orfs, prots = all_prots(strand, start='CTG')  # Teste com outro início específico
    self.assertIn('CTG', orfs)
    self.assertIn('L', prots)

    strand = "ATGATATAGCTGTAA"
    orfs, prots = all_prots(strand)  # Teste com múltiplos start
    self.assertIn('ATG', orfs)
    self.assertIn('CTG', orfs)
    self.assertIn('M', prots)
    self.assertIn('L', prots)
if __name__ == '__main__':
    suite = unittest.TestLoader().loadTestsFromTestCase(Test_ORFs)
    unittest.TextTestRunner(verbosity=3).run(suite)

test_all_rfs (__main__.Test_ORFs) ... ok
test_codon_table (__main__.Test_ORFs) ... ok
test_find_prots (__main__.Test_ORFs) ... ok
test_rf_codons (__main__.Test_ORFs) ... ok
test_start_stop_codons (__main__.Test_ORFs) ... ok
test_translate_rf (__main__.Test_ORFs) ... ok

----------------------------------------------------------------------
Ran 6 tests in 0.022s

OK


# Alinhamentos locais e globais

### **Alinhamentos globais:** Needleman-Wunsch

#### **Descrição do algoritmo**

O **Alinhamento Global (Needleman-Wunsch)** tem como objetivo fazer um alinhamento de duas sequências ao longo de toda a sua extensão. Para isso utiliza **programação dinâmica** para preencher uma matriz de pontuação com base numa pontuação de correspondência (quando os caracteres são semelhantes), penalidade de incompatibilidade (quando os caracteres diferem) e penalidade por gap (espaços inseridos para fazer o alinhamento).

#### **Projeto de alto nível**

**Entrada:**
Sequência 1 (seq1).
Sequência 2 (seq2).
Matriz de pontuação (ou pontuações fixas para correspondência, incompatibilidade e gap).


**Processamento:**
Iniciar a matriz de pontuação e de rastreamento.
Preencher a matriz com base nas regras do algoritmo global.
Rastrear o alinhamento a partir da matriz preenchida.


**Saída:**
Alinhamentos das sequências.
Pontuação total do alinhamento.

#### **Projeto de baixo nível**

Estruturas de Dados


Matriz de Pontuação:
Dimensões (len(seq1) + 1) x (len(seq2) + 1).
Armazena a melhor pontuação em cada posição.

Matriz de Rastreamento:
Armazena a direção do movimento (diagonal, para cima, ou para a esquerda).
'''
1. Iniciar a matriz de pontuação:
   Para cada célula da primeira linha e coluna, preencher com penalidades de gap.

2. Preencher a matriz de pontuação:
   Para cada célula (i, j):
      - Calcula o máximo entre:
         a) Diagonal (correspondência/incompatibilidade).
         b) Para cima (gap na sequência 2).
         c) Para a esquerda (gap na sequência 1).

3. Rastrear o alinhamento:
   Começa no canto inferior direito.
   Seguir as direções armazenadas na matriz de rastreamento até a origem.
'''

#### **Implementação**

In [2]:
def get_blosum62():
    blosum62_txt = """
   A  R  N  D  C  Q  E  G  H  I  L  K  M  F  P  S  T  W  Y  V  B  Z  X  *
A  4 -1 -2 -2  0 -1 -1  0 -2 -1 -1 -1 -1 -2 -1  1  0 -3 -2  0 -2 -1  0 -4
R -1  5  0 -2 -3  1  0 -2  0 -3 -2  2 -1 -3 -2 -1 -1 -3 -2 -3 -1  0 -1 -4
N -2  0  6  1 -3  0  0  0  1 -3 -3  0 -2 -3 -2  1  0 -4 -2 -3  3  0 -1 -4
D -2 -2  1  6 -3  0  2 -1 -1 -3 -4 -1 -3 -3 -1  0 -1 -4 -3 -3  4  1 -1 -4
C  0 -3 -3 -3  9 -3 -4 -3 -3 -1 -1 -3 -1 -2 -3 -1 -1 -2 -2 -1 -3 -3 -2 -4
Q -1  1  0  0 -3  5  2 -2  0 -3 -2  1  0 -3 -1  0 -1 -2 -1 -2  0  3 -1 -4
E -1  0  0  2 -4  2  5 -2  0 -3 -3  1 -2 -3 -1  0 -1 -3 -2 -2  1  4 -1 -4
G  0 -2  0 -1 -3 -2 -2  6 -2 -4 -4 -2 -3 -3 -2  0 -2 -2 -3 -3 -1 -2 -1 -4
H -2  0  1 -1 -3  0  0 -2  8 -3 -3 -1 -2 -1 -2 -1 -2 -2  2 -3  0  0 -1 -4
I -1 -3 -3 -3 -1 -3 -3 -4 -3  4  2 -3  1  0 -3 -2 -1 -3 -1  3 -3 -3 -1 -4
L -1 -2 -3 -4 -1 -2 -3 -4 -3  2  4 -2  2  0 -3 -2 -1 -2 -1  1 -4 -3 -1 -4
K -1  2  0 -1 -3  1  1 -2 -1 -3 -2  5 -1 -3 -1  0 -1 -3 -2 -2  0  1 -1 -4
M -1 -1 -2 -3 -1  0 -2 -3 -2  1  2 -1  5  0 -2 -1 -1 -1 -1  1 -3 -1 -1 -4
F -2 -3 -3 -3 -2 -3 -3 -3 -1  0  0 -3  0  6 -4 -2 -2  1  3 -1 -3 -3 -1 -4
P -1 -2 -2 -1 -3 -1 -1 -2 -2 -3 -3 -1 -2 -4  7 -1 -1 -4 -3 -2 -2 -1 -2 -4
S  1 -1  1  0 -1  0  0  0 -1 -2 -2  0 -1 -2 -1  4  1 -3 -2 -2  0  0  0 -4
T  0 -1  0 -1 -1 -1 -1 -2 -2 -1 -1 -1 -1 -2 -1  1  5 -2 -2  0 -1 -1  0 -4
W -3 -3 -4 -4 -2 -2 -3 -2 -2 -3 -2 -3 -1  1 -4 -3 -2 11  2 -3 -4 -3 -2 -4
Y -2 -2 -2 -3 -2 -1 -2 -3  2 -1 -1 -2 -1  3 -3 -2 -2  2  7 -1 -3 -2 -1 -4
V  0 -3 -3 -3 -1 -2 -2 -3 -3  3  1 -2  1 -1 -2 -2  0 -3 -1  4 -3 -2 -1 -4
B -2 -1  3  4 -3  0  1 -1  0 -3 -4  0 -3 -3 -2  0 -1 -4 -3 -3  4  1 -1 -4
Z -1  0  0  1 -3  3  4 -2  0 -3 -3  1 -1 -3 -1  0 -1 -3 -2 -2  1  4 -1 -4
X  0 -1 -1 -1 -2 -1 -1 -1 -1 -1 -1 -1 -1 -1 -2  0  0 -2 -1 -1 -1 -1 -1 -4
* -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4  1
"""

    header, *rest = [L.split() for L in blosum62_txt.splitlines() if L]

    return {L : {k : int(v) for k, v in zip(header, R)} for L, R in [(line[0], line[1:]) for line in rest]}



In [3]:
from pprint import pprint

def needleman_wunsch(seq1, seq2, g=-8):
    subst = get_blosum62()
    score = [[0 for x in '-' + seq1] for y in '-' + seq2]
    trace = [[' ' for x in '-' + seq1] for y in '-' + seq2]


    for p, x in enumerate(seq1):
        score[0][p + 1] = score[0][p] + g
        trace[0][p + 1] = 'E'
    for p, y in enumerate(seq2):
        score[p + 1][0] = score[p][0] + g
        trace[p + 1][0] = 'C'


    for p1, x1 in enumerate(seq1):
        for p2, x2 in enumerate(seq2):
            score[p2 + 1][p1 + 1] = max([
                    score[p2    ][p1    ] + subst[x1][x2],  # Diagonal
                    score[p2    ][p1 + 1] + g,              # Cima
                    score[p2 + 1][p1    ] + g,              # Esquerda
                    ]
                    )
            if score[p2 + 1][p1 + 1] == score[p2    ][p1    ] +subst[x1][x2]:
                trace[p2 + 1][p1 + 1] = 'D'
            if score[p2 + 1][p1 + 1] == score[p2    ][p1 + 1] + g:
                trace[p2 + 1][p1 + 1] = 'C'
            if score[p2 + 1][p1 + 1] == score[p2 + 1][p1    ] + g:
                trace[p2 + 1][p1 + 1] = 'E'

    return score, trace

def print_matrix(mat):
    for linha in mat:
        for x in linha:
            print(f"{x:3d}", end = " ")
        print()

def print_trace(mat):
    for linha in mat:
        print(*linha)

def score(seq1, seq2, g = -8):
    return needleman_wunsch(seq1, seq2, g)[-1][-1]

def reconstroi_align(seq1, seq2, trace):
    C = len(seq1)
    L = len(seq2)

    A1 = ''
    A2 = ''

    while C > 0 or L > 0:
        print(L, C)
        if trace[L][C] == 'D':
            L -= 1
            C -= 1
            A1 = seq1[C] + A1
            A2 = seq2[L] + A2
        elif trace[L][C] == 'E':
            C -= 1
            A1 = seq1[C] + A1
            A2 = '-' + A2
        elif trace[L][C] == 'C':
            L -= 1
            A1 = '-' + A1
            A2 = seq2[L] + A2
        else:
            raise ValueError("Unexpected trace value")
    print(A1)
    print(A2)
    return A1, A2

seq1 = input()
seq2 = input()
S, T = needleman_wunsch(seq1, seq2)
print_matrix(S)
print()
print_trace(T)

reconstroi_align(seq1, seq2, T)

AAGTAC
ATGTAC
  0  -8 -16 -24 -32 -40 -48 
 -8   4  -4 -12 -20 -28 -36 
-16  -4   4  -4  -7 -15 -23 
-24 -12  -4  10   2  -6 -14 
-32 -20 -12   2  15   7  -1 
-40 -28 -16  -6   7  19  11 
-48 -36 -24 -14  -1  11  28 

  E E E E E E
C D E E E E E
C C D E D E E
C C C D E E E
C C C C D E E
C C D C C D E
C C C C C C D
6 6
5 5
4 4
3 3
2 2
1 1
AAGTAC
ATGTAC


('AAGTAC', 'ATGTAC')

#### **Testes de unidade**

In [ ]:
import unittest

class TestAlignmentMethods(unittest.TestCase):

    def test_needleman_wunsch_basic(self):
        seq1 = "AGG"
        seq2 = "AGG"
        score, trace = needleman_wunsch(seq1, seq2, g=-8)

        # Verificando o valor de pontuação esperado para essas duas sequências
        self.assertEqual(score[len(seq2)][len(seq1)], 16)  # Pontuação esperada para alinhamento perfeito
        self.assertEqual(trace[len(seq2)][len(seq1)], 'D')  # Alinhamento diagonal

    def test_identical_sequences(self):
        seq1 = "AGT"
        seq2 = "AGT"
        S, T = needleman_wunsch(seq1, seq2)
        A1, A2 = reconstroi_align(seq1, seq2, T)
        self.assertEqual(A1, "AGT")
        self.assertEqual(A2, "AGT")

    def test_completely_different_sequences(self):
        seq1 = "AGT"
        seq2 = "CGA"
        S, T = needleman_wunsch(seq1, seq2)
        A1, A2 = reconstroi_align(seq1, seq2, T)
        self.assertEqual(A1, "AGT")
        self.assertEqual(A2, "CGA")

    def test_edge_case_empty_sequences(self):
        # Teste de sequências vazias
        seq1 = ""
        seq2 = ""

        S, T = needleman_wunsch(seq1, seq2)

        # Com ambas as sequências vazias, a pontuação deve ser 0
        self.assertEqual(S[-1][-1], 0)

        aligned_seq1, aligned_seq2 = reconstroi_align(seq1, seq2, T)

        # O alinhamento também deve ser vazio
        self.assertEqual(aligned_seq1, "")
        self.assertEqual(aligned_seq2, "")

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

### **Alinhamentos locais:** Smith-Waterman

#### **Descrição do algoritmo**

O **Alinhamento Local (Smith-Waterman)** tem como objetivo encontrar os segmentos de maior similaridade entre 2 sequências biológicas. Apesar de também utilizar **programação dinâmica**, difere do alinhamento de Needleman-Wunsch porque este permite alinhar apenas partes das sequências. Neste alinhamento apenas é retornado o segmento de maior pontuação.

**Passos do Algoritmo Smith-Waterman:**

    1º -> Iniciação:
            A matriz de pontuação começa com zeros. Não há penalidades iniciais para gaps.

    2º -> Preenchimento da Matriz:
            Cada célula é preenchida com o valor máximo entre:
            A pontuação de alinhamento diagonal anterior (correspondência/incompatibilidade).
            A pontuação da célula acima (gap na sequência 2).
            A pontuação da célula à esquerda (gap na sequência 1).
            Zero (opção de não alinhar).

    3º -> Identificação do Máximo:
            A célula com o maior valor na matriz representa o final do melhor alinhamento local.

    4º -> Rastreamento:
            A partir da célula de maior pontuação, seguimos as direções de rastreamento até encontrar uma célula com valor 0, recuperando as subsequências alinhadas.

#### **Projeto de alto nível**

**Entrada:** Sequências seq1 e seq2. Pontuação de match, penalidade de mismatch, penalidade de gap.

**Processamento:** Construir uma matriz de tamanho (len(seq1)+1) x (len(seq2)+1). Iniciar a matriz de pontuação com zeros. Preencher a matriz iterativamente com base na fórmula de pontuação. Identificar a célula com a maior pontuação. Rastrear o alinhamento local a partir dessa célula.

**Saída:** Melhor alinhamento local das duas sequências. Pontuação do alinhamento.A matriz de pontuação (opcional, para validação).

#### **Projeto de baixo nível**

Estruturas de Dados:

Matriz de Pontuação (score_matrix): Armazena as pontuações para cada par de posições.
Matriz de Rastreamento (traceback_matrix): Armazena a direção do movimento em cada posição (diagonal, para cima, para a esquerda).

Condições de Paragem:
O rastreamento termina quando a pontuação de uma célula atinge 0.

#### **Implementação**

In [4]:
from pprint import pprint

def smith_waterman(seq1, seq2, g=-8):
    subst = get_blosum62()
    score = [[0 for _ in '-' + seq1] for _ in '-' + seq2]
    trace = [[' ' for _ in '-' + seq1] for _ in '-' + seq2]

    # Preenchendo a matriz de pontuação
    for p1, x1 in enumerate(seq1):
        for p2, x2 in enumerate(seq2):
            # Calculando pontuação considerando diagonal (match/mismatch), cima e esquerda
            diag = score[p2][p1] + subst[x1][x2]
            cima = score[p2 + 1][p1] + g
            esquerda = score[p2][p1 + 1] + g

            # Pontuação máxima deve ser zero ou maior
            score[p2 + 1][p1 + 1] = max(0, diag, cima, esquerda)

            # Atribuindo as direções para backtracking
            if score[p2 + 1][p1 + 1] == diag:
                trace[p2 + 1][p1 + 1] = 'D'
            elif score[p2 + 1][p1 + 1] == cima:
                trace[p2 + 1][p1 + 1] = 'C'
            elif score[p2 + 1][p1 + 1] == esquerda:
                trace[p2 + 1][p1 + 1] = 'E'
            else:
                trace[p2 + 1][p1 + 1] = ' '  # Caso de zero, fim de uma possível sequência local

    return score, trace

def print_matrix(mat):
    for linha in mat:
        for x in linha:
            print(f"{x:3d}", end = " ")
        print()

def print_trace(mat):
    for linha in mat:
        print(*linha)

def score(seq1, seq2, g=-8):
    return smith_waterman(seq1, seq2, g)[-1][-1]

def reconstroi_align(seq1, seq2, trace):
    C = len(seq1)
    L = len(seq2)

    A1 = ''
    A2 = ''

    while C > 0 and L > 0 and trace[L][C] != ' ':
        print(L, C)
        if trace[L][C] == 'D':
            L -= 1
            C -= 1
            A1 = seq1[C] + A1
            A2 = seq2[L] + A2
        elif trace[L][C] == 'E':
            C -= 1
            A1 = seq1[C] + A1
            A2 = '-' + A2
        elif trace[L][C] == 'C':
            L -= 1
            A1 = '-' + A1
            A2 = seq2[L] + A2
        else:
            raise ValueError("Unexpected trace value")
    print(A1)
    print(A2)
    return A1, A2


S, T = smith_waterman(seq1, seq2)
print_matrix(S)
print()
print_trace(T)

A1, A2 = reconstroi_align(seq1, seq2, T)
print("Alinhamento final:")
print(A1)
print(A2)

  0   0   0   0   0   0   0 
  0   4   4   0   0   4   0 
  0   0   4   2   5   0   3 
  0   0   0  10   2   5   0 
  0   0   0   2  15   7   4 
  0   4   4   0   7  19  11 
  0   0   4   1   0  11  28 

             
  D D D D D D
  D D D D D D
  D D D C D  
  D D E D C D
  D D D E D C
  D D D   E D
6 6
5 5
4 4
3 3
2 2
1 1
AAGTAC
ATGTAC
Alinhamento final:
AAGTAC
ATGTAC


#### **Testes de unidade**

In [ ]:
import unittest

class TestSmithWaterman(unittest.TestCase):

    def test_identical_sequences(self):
        seq1 = "AGT"
        seq2 = "AGT"
        S, T = smith_waterman(seq1, seq2)
        A1, A2 = reconstroi_align(seq1, seq2, T)

        self.assertEqual(S[-1][-1], 3)  # A pontuação final deve ser 3
        self.assertEqual(A1, "AGT")
        self.assertEqual(A2, "AGT")

    def test_single_mismatch(self):
        seq1 = "AGT"
        seq2 = "AGC"
        S, T = smith_waterman(seq1, seq2)
        A1, A2 = reconstroi_align(seq1, seq2, T)

        self.assertEqual(S[-1][-1], 1)  # A pontuação final será 1 devido à substituição
        self.assertEqual(A1, "AG-")
        self.assertEqual(A2, "AGC")

    def test_gap_in_sequence(self):
        seq1 = "AGT"
        seq2 = "A--GT"
        S, T = smith_waterman(seq1, seq2)
        A1, A2 = reconstroi_align(seq1, seq2, T)

        self.assertEqual(S[-1][-1], 3)  # A pontuação final será 3
        self.assertEqual(A1, "AGT")
        self.assertEqual(A2, "A--GT")

    def test_no_match(self):
        seq1 = "AGT"
        seq2 = "XYZ"
        S, T = smith_waterman(seq1, seq2)
        A1, A2 = reconstroi_align(seq1, seq2, T)

        self.assertEqual(S[-1][-1], 0)  # A pontuação final será 0 pois não há correspondência
        self.assertEqual(A1, "")
        self.assertEqual(A2, "")

    def test_substitution_with_gap(self):
        seq1 = "AGT"
        seq2 = "AG-T"
        S, T = smith_waterman(seq1, seq2)
        A1, A2 = reconstroi_align(seq1, seq2, T)

        self.assertEqual(S[-1][-1], 3)  # A pontuação final será 3
        self.assertEqual(A1, "AGT")
        self.assertEqual(A2, "AG-T")

if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

# Blast

## Descrição do algoritmo

O BLAST (Basic Local Alignment Search Tool) é um dos algoritmos mais utilizados atualmente para pesquisas em bases de dados de sequências biológicas. O objetivo principal do BLAST é identificar alinhamentos locais de alta qualidade entre uma sequência de consulta (query) e as sequências presentes em uma base de dados.
O algoritmo funciona por meio de uma estratégia baseada em "palavras" curtas (seeds), que são subsequências extraídas da query. Essas seeds são comparadas com subsequências das sequências na base de dados para encontrar correspondências exatas ou quase exatas. Após identificar essas correspondências iniciais, o BLAST expande essas regiões em ambas as direções para formar alinhamentos maiores. O processo de expansão continua até que a pontuação do alinhamento caia abaixo de um valor pré-definido, conhecido como threshold.
Na implementação simplificada do BLAST apresentada aqui, o processo foi dividido em quatro etapas principais, cada uma realizada por uma função específica:

1.	query_map: Esta função realiza o mapeamento da query, gerando as seeds (subsequências) e suas posições na query.
2.	hits: Identifica as correspondências entre as seeds da query e as subsequências da sequência alvo na base de dados.
3.	extend_hit: Expande as correspondências identificadas pela função hits em ambas as direções, até que a pontuação do alinhamento caia abaixo do threshold.
4.	best_hit: Avalia todos os alinhamentos expandidos e seleciona aquele com a melhor pontuação, representando o melhor alinhamento encontrado entre a query e a sequência da base de dados.


## Projeto de alto nível

**query_map**

Esta função é responsável por dividir a sequência de consulta (query) em subsequências menores, chamadas de "seeds” (ou palavras), com tamanho definido pelo parâmetro w que representa a janela de procura. Cada seed gerada é armazenada como uma chave em um dicionário, onde o valor associado a cada chave é uma lista das posições em que essa seed ocorre na sequência de consulta. Essa estrutura de dados será usada nas etapas seguintes para identificar possíveis correspondências entre a query e as sequências da base de dados.

**hits**

A função hits utiliza o dicionário gerado pela função query_map e uma sequência alvo para identificar correspondências (hits) entre as palavras da query e a sequência alvo. Para cada chave no dicionário, a função encontra todas as posições em que esta aparece na sequência alvo e associa essas posições às posições correspondentes na query. O resultado final é uma lista de tuplos, onde cada tuplo contém dois valores: o índice da seed na query e o índice correspondente na sequência alvo.

**extend_hit**

Esta função recebe a query, a sequência alvo, o hit da função hits (um par de índices iniciais na query e na sequência alvo), e o parâmetro w. A função tenta estender o hit para os dois lados (esquerda e direita), desde que a proporção de correspondências seja de pelo menos 50% do tamanho da extensão.
A função devolve um tuplo contendo:
1.	O índice inicial do hit estendido na query;
2.	O índice inicial do hit estendido na sequência alvo;
3.	O tamanho total do alinhamento estendido;
4.	O número total de correspondências.

**best_hit**

A função best_hit combina as etapas anteriores para identificar o melhor alinhamento global entre a query e a sequência alvo. Ela utiliza as seeds geradas pela função query_map, identifica hits iniciais com a função hits, e expande esses hits com a função extend_hit. Entre todos os alinhamentos obtidos, é escolhido o melhor com base na maior pontuação global, que corresponde ao maior número de correspondências corretas no alinhamento estendido, se dois ou mais hits tiverem a mesma pontuação a função escolhe o que se encontra em primeiro lugar.




## Projeto de baixo nível

Projeto de baixo nível

Função query_map

Objetivo
A função query_map tem como objetivo criar um dicionário que associa subsequências (ou "seeds") da query às posições em que essas subsequências aparecem na query.
Entradas
1.	query (str): A sequência de consulta (query), que será dividida em subsequências.
2.	window_size (int): O tamanho da janela de procura, ou seja, o comprimento de cada subsequência (seed) gerada.
Saída
dict_seeds (defaultdict): Um dicionário onde:
•	As chaves são todas as subsequências (seeds) de tamanho window_size encontradas na query.
•	Os valores são listas com as posições iniciais em que cada seed aparece na query.
Passos Internos
1.	Criar um dicionário vazio (usando defaultdict) para armazenar as seeds e suas posições.
2.	Iterar sobre a sequência seq utilizando uma janela deslizante de tamanho window_size.
3.	Para cada subsequência extraída:
o	Adicionar a subsequência como chave no dicionário.
o	Adicionar a posição inicial da subsequência à lista correspondente no dicionário.
4.	Devolver o dicionário resultoante
Pseudocódigo
FUNÇÃO query_map(seq, window_size):
        Criar dicionário dict_seeds como defaultdict(lista)
        PARA cada posição i na query até (len(query) - window_size + 1):
                Extrair subsequência query[i : i + window_size]
                Adicionar i na lista associada à subsequência no dicionário dict_seeds
        Devolver dict_seeds

  
Função get_al_positions

Objetivo
A função get_all_positions localiza todas as posições iniciais de uma subsequência específica em uma sequência maior. É uma função auxiliar usada para encontrar correspondências exatas.
A função get_all_positions é fundamental para identificar correspondências exatas entre uma subsequência da query e a sequência alvo. Ela é usada pela função hits para criar pares de correspondência.

Entradas
1.	subseq (str): A subsequência que se deseja localizar na sequência alvo.
2.	seq (str): A sequência alvo onde será feita a busca.
Saída
•	positions (list[int]): Uma lista contendo todas as posições iniciais em que a subsequência ocorre na sequência alvo.

Passos Internos
1.	Definir uma lista vazia para armazenar as posições encontradas.
2.	Iterar sobre a sequência alvo com base no comprimento da subsequência.
3.	Comparar cada subsequência extraída com subseq:
o	Se houver correspondência, armazenar a posição inicial.
4.	Retornar a lista de posições encontradas.

Pseudocódigo

FUNÇÃO get_all_positions(subseq, seq):
        Criar lista vazia positions
        PARA cada posição i em seq até (len(seq) - len(subseq) + 1):
                Se seq[i : i + len(subseq)] for igual a subseq:
                        Adicionar i à lista positions
        Devolver a lista positions

Função hits

Objetivo
Esta função identifica todos os pares de posições correspondentes entre as subsequências (seeds) da query e a sequência alvo. Cada seed gerada pela função query_map é comparada com todas as suas ocorrências na sequência alvo, gerando uma lista de tuplos que representam os alinhamentos iniciais (hits).

Entradas
1.	qm (defaultdict): O dicionário gerado pela função query_map, onde:
o	As chaves são as seeds extraídas da query.
o	Os valores são listas de posições dessas seeds na query.
2.	seq (str): A sequência alvo onde os hits serão buscados.
Saídas
•	list[tuple[int, int]]:
Uma lista de tuplos, onde cada tuplo contém:
o	A posição inicial da seed na query (P1).
o	A posição inicial correspondente da mesma seed na sequência alvo (P2).

Passos Internos
1.	Definir uma lista vazia res para armazenar os hits encontrados.
2.	Iterar sobre cada seed (chave) e sua lista de posições (valores) no dicionário qm.
o	Para cada posição da seed na query (P1), buscar todas as posições da mesma seed na sequência alvo (P2) usando a função get_all_positions.
o	Para cada par de posições (P1, P2), adicionar o tuplo correspondente à lista res.
3.	Retornar a lista res contendo todos os hits encontrados.


Pseudocódigo

 FUNÇÃO hits(qm, seq):
      hits_l ← lista vazia
      PARA cada seed Q em qm:
            PARA cada posição P1 em qm[Q]:  # posições da seed na query
                  PARA cada posição P2 obtida por get_all_positions(Q, seq):  # posições da seed  
                        Adicionar o tuplo (P1, P2) à lista hits_l
   Devolver a lista hits_l


Função extend_hit

Objetivo
Expandir um hit inicial (alinhamento local) entre a query e a sequência alvo. A extensão ocorre em ambas as direções (esquerda e direita), enquanto a proporção de correspondências (matches) permanece acima de 50%.
________________________________________
Entradas
1.	query (str): A sequência de consulta (query).
2.	seq (str): A sequência alvo.
3.	hit (tuple[int, int]): Um tuplo representando um alinhamento inicial (hit):
o	O índice inicial da seed na query.
o	O índice inicial correspondente na sequência alvo.
4.	w (int): O tamanho da seed usada no alinhamento inicial.
________________________________________
Saídas
•	tuple[int, int, int, int]: Um tuplo contendo:
o	O índice inicial do alinhamento estendido na query.
o	O índice inicial do alinhamento estendido na sequência alvo.
o	O comprimento total do alinhamento estendido.
o	O número total de correspondências (matches) no alinhamento.
________________________________________

Passos Internos
1.	Definição:
o	Calcular os índices inicial (query_i, hit_i) e final (query_f, seq_f) do hit na query e na sequência alvo.
o	Estabelecer o número inicial de correspondências como o tamanho da seed (w).
o	Determinar se é possível expandir para a esquerda e direita verificando os limites das sequências.
2.	Extensão à Esquerda:
o	Enquanto houver espaço para expandir para a esquerda e a proporção de matches for suficiente (≥ 50%):
	Diminuir os índices iniciais (query_i e hit_i).
	Aumentar o comprimento do alinhamento.
	Comparar os caracteres da query e da sequência alvo nas novas posições:
	Aumentar o número de matches se os caracteres forem iguais.
o	Atualizar a condição de paragem.
3.	Extensão à Direita:
o	Enquanto houver espaço para expandir para a direita e a proporção de matches for suficiente (≥ 50%):
	Aumentar os índices finais (query_f e seq_f).
	Aumentar o comprimento do alinhamento.
	Comparar os caracteres da query e da sequência alvo nas novas posições:
	Aumentar o número de matches se os caracteres forem iguais.
o	Atualizar a condição de paragem.
4.	Devolver o resultado como um tuplo: os índices iniciais estendidos (query_i, hit_i), o comprimento do alinhamento estendido, e o número total de correspondências.
________________________________________







Pseudocódigo

FUNÇÃO extend_hit(query, seq, hit, w):
  Definir query_i e hit_i como os índices iniciais do hit na query e na sequência alvo,    respetivamente.
  Definir query_f e seq_f como os índices finais do hit na query e na sequência alvo, respetivamente.".
        correspondencias ← w
        extensao ← w
                Enquanto for possível expandir para a esquerda E correspondencias ≥ extensao / 2:
                                Diminuir query_i e hit_i.
                                Aumentar extensao.
                                Se query[query_i] == seq[hit_i]:
                                            Aumentar correspondencias.

            Enquanto for possível expandir para a direita E correspondencias ≥ extensao / 2:
                                Aumentar query_f e seq_f.
                                Aumentar extensao.
                                Se query[query_f] == seq[seq_f]:
                                            Aumentar correspondencias.

            Devolver (query_i, hit_i, query_f - query_i + 1, correspondencias)


---

Função best_hit
Objetivo
Identificar o melhor alinhamento entre uma query e uma sequência alvo. A função utiliza os resultados das funções query_map, hits e extend_hit para encontrar e avaliar alinhamentos locais. O melhor alinhamento é aquele com o maior número de correspondências (matches).
________________________________________
Entradas
1.	query (str): A sequência de consulta, que representa o padrão a ser alinhado.
2.	seq (str): A sequência alvo, onde buscamos os melhores alinhamentos.
3.	w (int): O tamanho da janela de procura, que determina o comprimento das seeds iniciais geradas.
________________________________________


Saídas
tuple[int, int, int, int]: Um tuplo contendo:
1.	Índice inicial na query: A posição inicial do melhor alinhamento na query.
2.	Índice inicial na sequência: A posição inicial do melhor alinhamento na sequência alvo.
3.	Tamanho do alinhamento: O comprimento do melhor alinhamento encontrado.
4.	Número de correspondências: A contagem de matches corretos no melhor alinhamento.
________________________________________
Passos Internos
1. Gerar o mapa de seeds
•	A função utiliza query_map para gerar um dicionário que mapeia as subsequências (seeds) da query e suas posições.
o	Entrada: A query e o tamanho da janela (w).
o	Saída: Um dicionário contendo cada seed como chave e as posições correspondentes como valores.
2. Encontrar hits iniciais
•	Com o dicionário gerado, a função chama hits para identificar todos os pares de posições correspondentes entre as seeds da query e a sequência alvo.
o	Entrada: O dicionário de seeds e a sequência alvo.
o	Saída: Uma lista de tuplos, onde cada tuplo contém a posição de uma seed na query e na sequência alvo.
3. Avaliar cada hit
•	Para cada hit da lista, a função chama extend_hit para expandi-lo, calculando o alinhamento completo.
o	Entrada: A query, a sequência alvo, o hit inicial e o tamanho da janela.
o	Saída: Um tuplo contendo os índices iniciais, o tamanho do alinhamento e o número total de correspondências.
4. Escolher o melhor hit
•	A função utiliza a função embutida max para selecionar o alinhamento com o maior número de correspondências.
o	Critério de desempate: Se dois hits tiverem o mesmo número de correspondências, o primeiro hit encontrado é priorizado.
o	Entrada: A lista de hits estendidos.
o	Saída: O melhor alinhamento representado por um tuplo com as informações do índice inicial, tamanho do alinhamento e número de matches.
________________________________________
Resumo do Funcionamento
A função best_hit combina as etapas principais do algoritmo BLAST:
1.	Gera seeds da query (query_map).
2.	Encontra hits iniciais entre a query e a sequência alvo (hits).
3.	Expande cada hit para maximizar o alinhamento (extend_hit).
4.	Seleciona o melhor alinhamento com base no número de correspondências.
Esta função sintetiza todo o processo e retorna o resultado final do alinhamento, que pode ser utilizado para análise biológica detalhada.


## Implementação

In [ ]:
from collections import defaultdict

def query_map(query, window_size):
    """
    Divide uma sequência em subsequências de tamanho fixo (seeds) e armazena as suas posições.
    Cada subsequência (seed) da query é usada como chave em um dicionário, e as posições
    em que a seed aparece na query são armazenadas em uma lista associada a essa chave.

    Args:
        seq (str): A sequência de consulta (query).
        window_size (int): O tamanho da janela de procura.

    Returns:
        dict : str, list[int]:
            Um dicionário onde as chaves são as seeds e os valores são listas com as posições em que cada seed ocorre na query.
            - chaves: são todas as subsequências (seeds) de tamanho w encontradas na query
            - valores: listas com todos as posições onde as seeds aparecem na query
    """
    if not isinstance(query, str):
        raise TypeError("A sequência deve ser uma string.")
    if not query.isalpha():
        raise ValueError("A sequência não pode conter caracteres inválidos.")
    if " " in query:
        raise ValueError("A sequência não pode conter espaços em branco.")
    if not isinstance(window_size, int) or window_size <= 0:
        raise ValueError("A janela de procura deve ser um número inteiro positivo.")
    if len(query) == 0:
        raise ValueError("A sequência não pode ser vazia.")

    dict_seed = defaultdict(list)
    size = len(query)
    for position in range(size - window_size + 1):
        seed = query[position : position + window_size]
        dict_seed[seed].append(position)
    return dict_seed


def get_all_positions(subseq, seq):
    """
    Encontra todas as posições em que uma subsequência ocorre em uma sequência maior.

    Args:
        subseq (str): A subsequência (seed) a ser procurada.
        seq (str): A sequência alvo onde a busca será realizada.

    Returns:
        list[int]: Uma lista de posições iniciais em que a subsequência ocorre na sequência alvo.
    """
    return [P for P in range(len(seq) - len(subseq) + 1) if seq[P : P + len(subseq)] == subseq]


def hits(qm, seq):
    """
    Identifica correspondências entre as seeds da query e a sequência alvo.
    Para cada seed gerada pela função `query_map`, a função encontra todas as
    posições correspondentes na sequência alvo.

    Args:
        qm (defaultdict): O dicionário gerado pela função `query_map` contendo as seeds e suas respectivas posições na query.
        seq (str): A sequência alvo onde as correspondências serão buscadas.

    Returns:
        list[tuple[int, int]]: Uma lista de tuplos, onde cada tuplo contém:
            - A posição inicial da seed na query;
            - A posição inicial correspondente na sequência alvo.
    """
    if not isinstance(seq, str):
        raise TypeError("A sequência deve ser uma string.")
    if not seq.isalpha():
        raise ValueError("A sequência não pode conter caracteres inválidos.")
    if seq == "":
        raise ValueError("A sequência não pode ser vazia.")
    if len(seq) == 0:
        raise ValueError("A sequência não pode ser vazia.")
    if not seq.isalpha():
        raise ValueError("A sequência não pode conter caracteres inválidos.")
    if " " in seq:
        raise ValueError("A sequência não pode conter espaços em branco.")


    hits_l = []
    for Q, LstPos in qm.items():
        for P1 in LstPos:
            for P2 in get_all_positions(Q, seq):
                hits_l.append((P1, P2))
    return hits_l


def extend_hit(query, seq, hit, w):
    """
    Esta função recebe uma query, uma sequência alvo, um hit e um tamanho de janela,
    e estende o hit na sequência até que não seja possível mais, mantendo um número mínimo de matches.

    Args:
        query (str): A sequência de consulta (query).
        seq (str): A sequência alvo.
        hit (tuple[int, int]): Uma tupla contendo os índices iniciais do hit na query e na sequência alvo.
        w (int): O tamanho da janela de procura.

    Returns:
        tuple[int, int, int, int]: Um tuplo contendo:
            - O índice inicial do hit estendido na query;
            - O índice inicial do hit estendido na sequência alvo;
            - O tamanho do alinhamento estendido;
            - O número total de correspondências no alinhamento estendido.
    """
    if not isinstance(query, str):
      raise TypeError("A sequência deve ser uma string")
    if not query.isalpha():
      raise ValueError("A query não pode conter caracteres inválidos.")
    if not seq.isalpha():
      raise ValueError("A sequência não pode conter caracteres inválidos.")
    if len(query) == 0 or len(seq) == 0:
      raise ValueError("A sequência não pode ser vazia")
    if not isinstance(seq, str):
      raise TypeError("A sequência deve ser uma string")
    if not isinstance(w, int) or w <= 0:
      raise ValueError("O tamanho da janela deve ser um inteiro positivo")

    query_i = hit[0]
    hit_i = hit[1]
    query_f = hit[0] + w - 1
    seq_f = hit[1] + w - 1

    correspondencias = w
    extensao = w
    espaço_esquerda = query_i > 0 and hit_i > 0
    espaço_direita = query_f < len(query) - 1 and seq_f < len(seq) - 1

    while correspondencias >= extensao / 2 and espaço_esquerda:
        query_i -= 1
        hit_i -= 1
        extensao += 1
        if query[query_i] == seq[hit_i]:
            correspondencias += 1
        espaço_esquerda = query_i > 0 and hit_i > 0

    while correspondencias >= extensao / 2 and espaço_direita:
        query_f += 1
        seq_f += 1
        extensao += 1
        if query[query_f] == seq[seq_f]:
            correspondencias += 1
        espaço_direita = query_f < len(query) - 1 and seq_f < len(seq) - 1

    return (query_i, hit_i, query_f - query_i + 1, correspondencias)


def best_hit(query: str, seq: str, w: int) -> tuple[int, int, int, int]:
    """
    Identifica o melhor alinhamento entre uma query e uma sequência alvo.
    A função utiliza as funções `query_map`, `hits`, e `extend_hit` para encontrar
    e avaliar alinhamentos locais entre a query e a sequência alvo. O melhor alinhamento
    é determinado com base no maior número de correspondências (matches).

    Args:
        query (str): A sequência de consulta (query).
        seq (str): A sequência alvo.
        w (int): O tamanho da janela de procura.

    Returns:
        tuple[int, int, int, int]: Uma tupla contendo:
            - O índice inicial do melhor hit na query;
            - O índice inicial do melhor hit na sequência alvo;
            - O tamanho do alinhamento do melhor hit;
            - O número total de correspondências no melhor hit.
    """
    if not isinstance(query, str):
      raise TypeError("A sequência deve ser uma string")
    if not isinstance(seq, str):
      raise TypeError("A sequência deve ser uma string")
    if not isinstance(w, int) or w <= 0:
      raise ValueError("O tamanho da janela deve ser um inteiro positivo")

    qm = query_map(query, w)
    hits_list = hits(qm, seq)

    def hit_score(hit):
        extended_hit = extend_hit(query, seq, hit, w)
        return extended_hit[3], -hits_list.index(hit), extended_hit

    _, _, best_hit_result = max((hit_score(hit) for hit in hits_list), default=(0, 0, None))
    return best_hit_result


In [ ]:
#Teste
query = "AATATAT"
seq = "AATATGTTATATAATAATATTT"
w = 3
best_hit(query, seq, w)

(0, 0, 7, 6)

## Testes de unidade

### Query_map

In [ ]:
import unittest

class TestBlast(unittest.TestCase):
    def test_query_map_vazia(self):
        # Testa se a função lança um erro ao receber uma sequência vazia
        with self.assertRaises(ValueError):
            query_map("", 3)

    def espaço(self):
      with self.assertRaises(ValueError):
          query_map("ATC G", 3)  # Testa espaços em branco

    def test_query_map_w_negativo(self):
        # Testa se a função lança um erro ao receber um valor negativo para w
        with self.assertRaises(ValueError):
            query_map("ACT", -3)

    def test_query_map_w_number(self):
        # Testa se a função lança um erro ao receber um valor não string para a query
        with self.assertRaises(TypeError):
            query_map(66//5, "3")

    def test_query_map_output(self):
        # Testa se a função dá o resultado esperado
        output = {'ATG': [0], 'TGC': [1], 'GCG': [2], 'CGG': [3], 'GGT': [4]}
        self.assertEqual(query_map("ATGCGGT", 3), output)

    def test_query_map_output_false(self):
        # Testa se a função dá o erro para o resultado não esperado para esses parâmetros
        output = {'AAT': [0], 'ATA': [1, 4], 'TAT': [2, 3]}
        self.assertFalse(query_map("AATATAT", 3) == output)

suite = unittest.TestLoader().loadTestsFromTestCase(TestBlast)
unittest.TextTestRunner(verbosity=3).run(suite)

test_query_map_output (__main__.TestBlast.test_query_map_output) ... ok
test_query_map_output_false (__main__.TestBlast.test_query_map_output_false) ... ok
test_query_map_vazia (__main__.TestBlast.test_query_map_vazia) ... ok
test_query_map_w_negativo (__main__.TestBlast.test_query_map_w_negativo) ... ok
test_query_map_w_number (__main__.TestBlast.test_query_map_w_number) ... ok

----------------------------------------------------------------------
Ran 5 tests in 0.019s

OK


<unittest.runner.TextTestResult run=5 errors=0 failures=0>

### hits

In [ ]:
qm = query_map("AATATAT", 3)
class TestBlast(unittest.TestCase):
    def test_hit_vazia(self):
        # Testa se a função lança um erro ao receber uma sequência vazia
        with self.assertRaises(ValueError):
            hits(qm, "")

    def test_hit_w_number(self):
        # Testa se a função lança um erro ao receber um valor não string para a sequência
        with self.assertRaises(TypeError):
            hits(qm, 34)

    def test_hit_invalida(self):
        # Testa se a função lança um erro ao receber uma sequência com carateres inválidas
        with self.assertRaises(ValueError):
            hits(qm, "3%$&/sw")

    def test_hit_output(self):
        # Testa se a função dá o resultado esperado para esses parâmetros
        q = query_map("AATATAT", 3)
        output = [(0, 0), (0, 12), (0, 15), (1, 1), (1, 8), (1, 10), (1, 13), (1, 16), (3, 1), (3, 8), (3, 10), (3, 13), (3, 16), (2, 2), (2, 7), (2, 9), (2, 17), (4, 2), (4, 7), (4, 9), (4, 17)]
        self.assertEqual(hits(q, "AATATGTTATATAATAATATTT"), output)

    def test_hit_output_false(self):
        # Testa se a função dá o erro para o resultado não esperado para esses parâmetros
        output = [(0, 0), (0, 12), (0, 11), (1, 1), (1, 9), (1, 7), (1, 13), (1, 16), (3, 1), (3, 8), (3, 10), (3, 13), (3, 16), (2, 2), (2, 7), (2, 9), (2, 17), (4, 2), (4, 7), (4, 9), (4, 17)]
        q = query_map("AATATAT", 3)
        self.assertFalse(hits(q, "AATATGTTATATAATAATATTT") == output)

suite = unittest.TestLoader().loadTestsFromTestCase(TestBlast)
unittest.TextTestRunner(verbosity=3).run(suite)

test_hit_invalida (__main__.TestBlast.test_hit_invalida) ... ok
test_hit_output (__main__.TestBlast.test_hit_output) ... ok
test_hit_output_false (__main__.TestBlast.test_hit_output_false) ... ok
test_hit_vazia (__main__.TestBlast.test_hit_vazia) ... ok
test_hit_w_number (__main__.TestBlast.test_hit_w_number) ... ok

----------------------------------------------------------------------
Ran 5 tests in 0.016s

OK


<unittest.runner.TextTestResult run=5 errors=0 failures=0>

### extend_hits

In [ ]:
class TestBlast(unittest.TestCase):
    def test_extend_hit_seq_vazia(self):
        # Testa se a função lança um erro ao receber uma sequência vazia
        with self.assertRaises(ValueError):
            extend_hit("AATATAT", "", (0, 12), 3)

    def test_extend_hit_query_vazia(self):
        # Testa se a função lança um erro ao receber uma query vazia
        with self.assertRaises(ValueError):
            extend_hit("", "AATATGTTATATAATAATATTT", (0, 12), 3)

    def test_extend_hit_w_negativo(self):
        # Testa se a função lança um erro ao receber um valor negativo para w
        with self.assertRaises(ValueError):
            extend_hit("AATATAT", "AATATGTTATATAATAATATTT", (0, 12), -9)

    def test_extend_hit_w_number_query(self):
        # Testa se a função lança um erro ao receber um valor não string para a query
        with self.assertRaises(TypeError):
            extend_hit(33, "AATATGTTATATAATAATATTT", (0, 12), 3)

    def test_extend_hit_seq_invalida(self):
        # Testa se a função lança um erro ao receber uma sequência com letras inválidas
        with self.assertRaises(ValueError):
            extend_hit("AATATAT", "SW/iopgF", (0, 12), 3)

    def test_extend_hit_query_invalida(self):
        # Testa se a função lança um erro ao receber uma query com letras inválidas
        with self.assertRaises(ValueError):
            extend_hit("qwi//ofgr", "AATATGTTATATAATAATATTT", (0, 12), 3)

    def test_extend_hit_output(self):
        # Testa se a função dá o resultado esperado para esses parâmetros
        output = (0, 12, 7, 4)
        self.assertEqual(extend_hit("AATATAT", "AATATGTTATATAATAATATTT", (0, 12), 3), output)

    def test_extend_hit_output_false(self):
        # Testa se a função dá erro para o resultado não esperado para esses parâmetros
        output = (0, 12, 4, 3)
        self.assertFalse(extend_hit("AATATAT", "AATATGTTATATAATAATATTT", (0, 12), 3) == output)

suite = unittest.TestLoader().loadTestsFromTestCase(TestBlast)
unittest.TextTestRunner(verbosity=3).run(suite)

test_extend_hit_output (__main__.TestBlast.test_extend_hit_output) ... ok
test_extend_hit_output_false (__main__.TestBlast.test_extend_hit_output_false) ... ok
test_extend_hit_query_invalida (__main__.TestBlast.test_extend_hit_query_invalida) ... ok
test_extend_hit_query_vazia (__main__.TestBlast.test_extend_hit_query_vazia) ... ok
test_extend_hit_seq_invalida (__main__.TestBlast.test_extend_hit_seq_invalida) ... ok
test_extend_hit_seq_vazia (__main__.TestBlast.test_extend_hit_seq_vazia) ... ok
test_extend_hit_w_negativo (__main__.TestBlast.test_extend_hit_w_negativo) ... ok
test_extend_hit_w_number_query (__main__.TestBlast.test_extend_hit_w_number_query) ... ok

----------------------------------------------------------------------
Ran 8 tests in 0.049s

OK


<unittest.runner.TextTestResult run=8 errors=0 failures=0>

### best_hit

In [ ]:
class TestBlast(unittest.TestCase):
    def test_best_hit_seq_vazia(self):
        # Testa se a função lança um erro ao receber uma sequência vazia
        with self.assertRaises(ValueError):
            best_hit("AATATAT", "", 3)

    def test_best_hit_query_vazia(self):
        # Testa se a função lança um erro ao receber uma query vazia
        with self.assertRaises(ValueError):
            best_hit("", "AATATGTTATATAATAATATTT", 3)

    def test_best_hit_w_negativo(self):
        # Testa se a função lança um erro ao receber um valor negativo para w
        with self.assertRaises(ValueError):
            best_hit("AATATAT", "AATATGTTATATAATAATATTT", -3)

    def test_best_hit_w_number_seq(self):
        # Testa se a função lança um erro ao receber um valor não string para a sequência
        with self.assertRaises(TypeError):
            best_hit("AATATAT", 33, 3)

    def test_best_hit_w_number_query(self):
        # Testa se a função lança um erro ao receber um valor não string para a query
        with self.assertRaises(TypeError):
            best_hit(33, "AATATGTTATATAATAATATTT", 3)

    def test_best_hit_seq_invalida(self):
        # Testa se a função lança um erro ao receber uma sequência com letras inválidas
        with self.assertRaises(ValueError):
            best_hit("AATATAT", "SWiop1gF", 3)

    def test_best_hit_query_invalida(self):
        # Testa se a função lança um erro ao receber uma query com letras inválidas
        with self.assertRaises(ValueError):
            best_hit("qw/iofgr", "AATATGTTATATAATAATATTT", 3)

    def test_best_hit_output(self):
        # Testa se a função dá o resultado esperado para esses parâmetros
        output = (0, 0, 7, 6)
        self.assertEqual(best_hit("AATATAT", "AATATGTTATATAATAATATTT", 3), output)

    def test_best_hit_output_false(self):
        # Testa se a função dá erro para o resultado não esperado para esses parâmetros
        output = (0, 0, 5, 6)
        self.assertFalse(best_hit("AATATAT", "AATATGTTATATAATAATATTT", 3) == output)

suite = unittest.TestLoader().loadTestsFromTestCase(TestBlast)
unittest.TextTestRunner(verbosity=3).run(suite)

test_best_hit_output (__main__.TestBlast.test_best_hit_output) ... ok
test_best_hit_output_false (__main__.TestBlast.test_best_hit_output_false) ... ok
test_best_hit_query_invalida (__main__.TestBlast.test_best_hit_query_invalida) ... ok
test_best_hit_query_vazia (__main__.TestBlast.test_best_hit_query_vazia) ... ok
test_best_hit_seq_invalida (__main__.TestBlast.test_best_hit_seq_invalida) ... ok
test_best_hit_seq_vazia (__main__.TestBlast.test_best_hit_seq_vazia) ... ok
test_best_hit_w_negativo (__main__.TestBlast.test_best_hit_w_negativo) ... ok
test_best_hit_w_number_query (__main__.TestBlast.test_best_hit_w_number_query) ... ok
test_best_hit_w_number_seq (__main__.TestBlast.test_best_hit_w_number_seq) ... ok

----------------------------------------------------------------------
Ran 9 tests in 0.031s

OK


<unittest.runner.TextTestResult run=9 errors=0 failures=0>

# Motifs

#Descrição do algoritmo
O algoritmo analisa sequências de DNA (ou proteínas) para encontrar motifs mais prováveis numa sequência alvo com base numa PWM (Position Weight Matrix), posteriormente converte os valores da PWM para PSSM (Position Specific Scoring Matrix) e irá funcionar em quatro etapas principais:

###1) Construção da tabela de contagens
O algoritmo conta a ocorrência de cada base em cada posição das sequências fornecidas. Para garantir que não haja problemas com valores nulos é aplicada uma pseudocontagem somando um valor mínimo a todas as contagens.
###2) Normalização para PWM
A tabela de contagens gerada é transformada numa PWM, onde as contagens são convertidas em probabilidades. Cada posição da matriz representa a probabilidade de ocorrência de uma base específica (A, C, G, T) naquela posição das sequências, levando em consideração a totalidade das sequências fornecidas.
###3) Cálculo da PSSM
A partir da PWM é calculada a PSSM. A PSSM representa o logaritmo da razão entre a probabilidade observada de cada símbolo numa posição (como calculado na PWM) e a frequência de fundo esperada para aquele símbolo, utilizando logaritmos de base 2. A PSSM transforma a PWM numa métrica de score, onde valores positivos indicam que uma base é mais provável do que a frequência de fundo, e valores negativos indicam o oposto.
###4) Procura do Motivo Mais Provável
Com a PWM e PSSM calculadas, o algoritmo percorre a sequência alvo para identificar as subsequências de comprimento igual ao da PWM ou PSSM. Para cada subsequência, o algoritmo calcula sua probabilidade ou score, dependendo se utiliza a PWM ou a PSSM. As subsequências com as maiores probabilidades (PWM) ou maiores scores (PSSM) são consideradas como os motifs mais prováveis na sequência alvo.

Esse processo permite que o algoritmo identifique com precisão as regiões da sequência alvo que mais se assemelham aos motivos de interesse, utilizando tanto a PWM para avaliar probabilidades como a PSSM para avaliar scores com base na razão das probabilidades, ajustada à frequência de fundo.

#Descrição de alto nível

##Input:

**Conjunto de Sequências:** Um conjunto de sequências biológicas que são utilizadas para construir a PWM (Position Weight Matrix).

**Sequência Alvo:** A sequência na qual o motif mais provável será procurado.

**Parâmetros Opcionais:** Pseudocontagem, que ajuda a evitar valores nulos nas matrizes.

##Processo:

**Construção da Tabela de Contagens:**
A função recebe as sequências e calcula a ocorrência de cada base em cada posição nas sequências. Uma pseudocontagem pode ser somada para evitar zero como valor inicial em qualquer posição da matriz de contagens.

**Geração da PWM:** A PWM é gerada a partir da tabela de contagens. As frequências observadas para cada base nas diferentes posições das sequências são normalizadas, o que resulta numa matriz de probabilidades de cada base para cada posição.

**Cálculo da PSSM:** A PSSM (Position Specific Scoring Matrix) é gerada a partir da PWM. Utilizando o logaritmo da razão entre a probabilidade observada e a probabilidade esperada (background), a PSSM fornece uma medida de "afinidade" de cada base para cada posição, usando uma escala logarítmica.

**Cálculo das Probabilidades e Scores de Subsequentemente:** Para cada subsequência da sequência alvo com o mesmo comprimento da PWM ou PSSM, calculam-se as probabilidades (para PWM) ou os scores (para PSSM) correspondentes.

**Identificação dos Motifs Mais Prováveis:** A subsequência com a maior probabilidade ou score será considerada o motif mais provável. O código identifica essas subsequências ao avaliar todas as subsequências possíveis, comparando suas probabilidades (PWM) ou scores (PSSM), e retorna as subsequências que apresentam os maiores valores.

##Saída:

**Matriz PWM:** A Position Weight Matrix, que contém as probabilidades normalizadas de cada base para cada posição das sequências.

**Matriz PSSM:** A Position Specific Scoring Matrix, que contém os scores logaritmos baseados nas probabilidades da PWM.

**Motif(s) Mais Provável(eis):** A(s) subsequência(s) mais provável(is) encontrada(s) na sequência alvo com base na PWM e/ou PSSM

In [ ]:
import math
import re

seqs = ['ATTGACTGACTA','ATCGTGACTGAC','ATTGACTGACTC','TGACTGACACTC']

'''Recebe as sequências e um alfabeto. Para cada posição nas sequências, conta a ocorrência de cada símbolo, somando a pseudocontagem.'''
def tabela_contagens(seqs, alfabeto = "ACGT", pseudocontagem = 1):
    for seq in seqs:
        for base in seq:
            if base not in alfabeto:
                raise AssertionError(f"Caractere inválido '{base}' encontrado na sequência.") # Verifica se todas as bases nas sequências estão dentro do alfabeto permitido
    return [{b: occ.count(b) + pseudocontagem for b in alfabeto} for occ in zip(*seqs)] # Calcula a contagem de cada base nas diferentes posições das sequências e retorna uma lista de dicionários

'''Normaliza a tabela de contagens dividindo cada valor pelo número total de sequências ajustado à pseudocontagem'''
def pwm(seqs, tipo = "DNA", pseudocontagem = 0):
    assert all(len(seqs[0]) == len(s) for s in seqs) # Verifica se todas as sequências têm o mesmo comprimento

    '''Define o tipo de sequência (DNA ou Proteína) e o alfabeto apropriado.'''
    tipo = tipo.upper()
    assert tipo in "DNA PROTEIN".split(), f"Tipo {tipo} inválido!"
    alfabeto = "ACGT" if tipo == "DNA" else "ACGU"
    '''Verifica se todas as bases nas sequências estão dentro do alfabeto especificado.'''
    for seq in seqs:
        for base in seq:
            if base not in alfabeto:
                raise AssertionError(f"Caractere inválido '{base}' encontrado na sequência.")

    tabela = tabela_contagens(seqs, alfabeto=alfabeto, pseudocontagem=pseudocontagem) # Calcula a tabela de contagens

    '''Normaliza a tabela de contagens para gerar a PWM'''
    L = len(seqs)
    A = len(alfabeto)

    return [{k: v / (L + A * pseudocontagem) for k, v in linha.items()} for linha in tabela] # Retorna a PWM normalizada

'''Imprime a PWM com um número específico de casas decimais.'''
def imprime_pwm(pwm, casas_decimais = 2):
    return [{k : round(v, casas_decimais) for k, v in linha.items()} for linha in pwm]

'''Calcula a PSSM a partir de uma PWM, utilizando logaritmos baseados na razão das probabilidades'''
def pssm(pwm, bg_freq=None):
    if bg_freq is None:
        bg_freq = {k: 1 / len(pwm[0]) for k in pwm[0].keys()} # Frequência uniforme para cada base

    if not all(isinstance(coluna, dict) for coluna in pwm):
        raise AssertionError("A PWM deve ser uma lista de dicionários.") # Verifica se a PWM é uma lista de dicionários

    return [
        {k: math.log2(v / bg_freq[k]) if v > 0 else float('-inf') for k, v in coluna.items()} # Calcula a PSSM para cada coluna da PWM através do logaritmo das probabilidades
        for coluna in pwm
    ]

'''Imprime a PSSM com casas decimais.'''
def imprime_pssm(pssm, casas_decimais=2):
    return [{k: round(v, casas_decimais) for k, v in linha.items()} for linha in pssm]

'''Calcula a probabilidade de uma sequência com base na PWM, multiplicando as probabilidades das letras correspondentes nas posições respetivas'''
def prob_gerar_sequencia(seq, pwm):
    assert len(seq) == len(pwm) # Verifica se o comprimento da sequência é compatível com a PWM

    prob = 1
    for letra, coluna in zip(seq, pwm):
        prob *= coluna[letra] # Multiplica as probabilidades das bases correspondentes
    return prob

'''Calcula o score de uma sequência com base na PSSM, somando os scores das letras correspondentes nas posições respetivas'''
def score_gerar_sequencia(seq, pssm):
    assert len(seq) == len(pssm) # Verifica se o comprimento da sequência é compatível com a PSSM

    score = 0
    for letra, coluna in zip(seq, pssm):
        score += coluna[letra] # Soma os scores das bases correspondentes
    return score

'''Percorre a sequência alvo e avalia todas as subsequências possíveis do tamanho da PWM ou PSSM. Calcula os scores ou probabilidades de todas as subsequências e retorna a(s) subsequência(s) com o maior valor.'''
def seq_mais_provavel(seq, matriz, tipo="PWM"):
    assert len(seq) >= len(matriz) # Verifica se a sequência é longa o suficiente para ter subsequências
    L = len(matriz) # Tamanho da matriz (tamanho da subsequência)

    R = '.' * L # Expressão regular para encontrar todas as subsequências possíveis
    resultados = {}
    for s in re.findall(f'(?=({R}))', seq): # Percorre todas as subsequências
        if tipo == "PWM":
            resultados[s] = prob_gerar_sequencia(s, matriz) # Calcula a probabilidade para PWM
        elif tipo == "PSSM":
            resultados[s] = score_gerar_sequencia(s, matriz) # Calcula a probabilidade para PSSM

    '''Encontra o maior valor de probabilidade/score e retorna as subsequências correspondentes ordenadas de forma crescente'''
    maior_valor = max(resultados.values())
    return sorted(set([k for k, v in resultados.items() if v >= maior_valor]))

'''Cálculo da PWM'''
pwm_result = pwm(seqs, tipo="DNA", pseudocontagem=1)
print("PWM:")
print(imprime_pwm(pwm_result))

'''Cálculo da PSSM com base no resultado da PWM'''
pssm_result = pssm(pwm_result)
print("\nPSSM:")
print(imprime_pssm(pssm_result))

'''Exemplo de uma sequência a testar'''
seq_a_testar = "ATTGACTCATTC"

'''Encontra as subsequências mais prováveis de acordo com a PWM'''
subseqs_mais_provaveis_pwm = seq_mais_provavel(seq_a_testar, pwm_result, tipo="PWM")
'''Encontra as subsequências mais prováveis de acordo com a PSSM'''
subseqs_mais_provaveis_pssm = seq_mais_provavel(seq_a_testar, pssm_result, tipo="PSSM")

'''Demostração dos resultados'''
print("\nSubsequências mais prováveis (PWM):")
print(subseqs_mais_provaveis_pwm)

print("\nSubsequências com maior score (PSSM):")
print(subseqs_mais_provaveis_pssm)

PWM:
[{'A': 0.5, 'C': 0.12, 'G': 0.12, 'T': 0.25}, {'A': 0.12, 'C': 0.12, 'G': 0.25, 'T': 0.5}, {'A': 0.25, 'C': 0.25, 'G': 0.12, 'T': 0.38}, {'A': 0.12, 'C': 0.25, 'G': 0.5, 'T': 0.12}, {'A': 0.38, 'C': 0.12, 'G': 0.12, 'T': 0.38}, {'A': 0.12, 'C': 0.38, 'G': 0.38, 'T': 0.12}, {'A': 0.38, 'C': 0.12, 'G': 0.12, 'T': 0.38}, {'A': 0.12, 'C': 0.38, 'G': 0.38, 'T': 0.12}, {'A': 0.5, 'C': 0.12, 'G': 0.12, 'T': 0.25}, {'A': 0.12, 'C': 0.5, 'G': 0.25, 'T': 0.12}, {'A': 0.25, 'C': 0.12, 'G': 0.12, 'T': 0.5}, {'A': 0.25, 'C': 0.5, 'G': 0.12, 'T': 0.12}]

PSSM:
[{'A': 1.0, 'C': -1.0, 'G': -1.0, 'T': 0.0}, {'A': -1.0, 'C': -1.0, 'G': 0.0, 'T': 1.0}, {'A': 0.0, 'C': 0.0, 'G': -1.0, 'T': 0.58}, {'A': -1.0, 'C': 0.0, 'G': 1.0, 'T': -1.0}, {'A': 0.58, 'C': -1.0, 'G': -1.0, 'T': 0.58}, {'A': -1.0, 'C': 0.58, 'G': 0.58, 'T': -1.0}, {'A': 0.58, 'C': -1.0, 'G': -1.0, 'T': 0.58}, {'A': -1.0, 'C': 0.58, 'G': 0.58, 'T': -1.0}, {'A': 1.0, 'C': -1.0, 'G': -1.0, 'T': 0.0}, {'A': -1.0, 'C': 1.0, 'G': 0.0, 'T': 

#Testes Unitários

In [ ]:
import unittest

class TestMotifs(unittest.TestCase):

  '''Verifica se a função pwm lança um erro quando o tipo inválido é passado.'''

  def test_alfabeto_caracter_errado_dna(self):
    with self.assertRaises(AssertionError):
      pwm(['ATTG', 'ATCG','ATTC','ACTC'], tipo="DNA%")

  '''Verifica se a função tabela_contagens emite erro quando há caracteres inválidos nas sequências de DNA.'''

  def test_tabela_contagens_caracter_invalido_dna(self):
    seqs_invalidas = ['ATTX', 'ATCG', 'ACGT', 'ACTC']
    with self.assertRaises(AssertionError):
        tabela_contagens(seqs_invalidas, alfabeto="ACGT")

  '''Verifica se a função tabela_contagens emite erro quando há caracteres inválidos nas sequências de RNA.'''

  def test_tabela_contagens_caracter_invalido_rna(self):
    seqs_invalidas = ['AUGC', 'ACUG', 'AGCU', 'ACTC']
    with self.assertRaises(AssertionError):
        tabela_contagens(seqs_invalidas, alfabeto="ACGU")

  '''Verifica se a função pwm lança um erro quando as sequências têm tamanhos diferentes.'''

  def test_pwm_tamanhos_diferentes(self):
    with self.assertRaises(AssertionError):
        pwm(["ACGT", "TGCA", "ACG"], "DNA")

  ''' Verifica se a função imprime_pwm gera a matriz PWM correta e a exibe com a quantidade adequada de casas decimais.'''

  def test_imprime_pwm(self):
        pwm_matrix = pwm(["ATTG", "ATCG"], tipo="DNA", pseudocontagem=1)
        result = imprime_pwm(pwm_matrix, casas_decimais=2)
        expected = [
            {'A': 0.5, 'C': 0.17, 'G': 0.17, 'T': 0.17},
            {'A': 0.17, 'C': 0.17, 'G': 0.17, 'T': 0.5},
            {'A': 0.17, 'C': 0.33, 'G': 0.17, 'T': 0.33},
            {'A': 0.17, 'C': 0.17, 'G': 0.5, 'T': 0.17}
            ]
        self.assertEqual(result, expected)

  def test_seq_mais_provavel_pwm(self):
        pwm_matrix = pwm(["ATTG", "ATCG"], tipo="DNA", pseudocontagem=1)
        result = seq_mais_provavel("ATTGACTCATTC", pwm_matrix, tipo="PWM")
        self.assertIn("ATTG", result)

  '''Verificar que a função pssm() gera um erro de AssertionError quando as sequências fornecidas à função pwm() têm tamanhos diferentes.'''

  def test_pssm_tamanhos_de_sequencia_diferentes(self):
    seqs_invalidas = ["ACT", "TGCA", "TCG"]
    with self.assertRaises(AssertionError):
      pwm(seqs_invalidas, "DNA")

  '''Verifica se a PSSM é composta por caracteres inválidos.'''

  def test_pssm_caracter_invalido(self):
        seqs_invalidas = ("AC/", "UGC", "UCG")
        with self.assertRaises(AssertionError):
            pssm(seqs_invalidas, "RNA")

  '''Verifica se a PSSM está a retornar corretamente a sequência mais provável.'''

  def test_seq_mais_provavel_pssm(self):
        pwm_matrix = pwm(["ATTG", "ATCG"], tipo="DNA", pseudocontagem=1)
        pssm_matrix = pssm(pwm_matrix)
        result = seq_mais_provavel("ATTGACTCATTC", pssm_matrix, tipo="PSSM")
        self.assertIn("ATTG", result)

  '''Verifica se todos os valores retornados pela matriz de PSSM são floats.'''

  def test_pssm_with_custom_bg_freq(self):
        pwm_matrix = pwm(["ATTG", "ATCG"], tipo="DNA", pseudocontagem=1)
        bg_freq = {'A': 0.4, 'C': 0.2, 'G': 0.2, 'T': 0.2}
        result = pssm(pwm_matrix, bg_freq=bg_freq)
        self.assertTrue(all(isinstance(v, float) for col in result for v in col.values()))

suite = unittest.TestLoader().loadTestsFromTestCase(TestMotifs)
unittest.TextTestRunner(verbosity=3).run(suite)

test_alfabeto_caracter_errado_dna (__main__.TestMotifs.test_alfabeto_caracter_errado_dna) ... ok
test_imprime_pwm (__main__.TestMotifs.test_imprime_pwm) ... ok
test_pssm_caracter_invalido (__main__.TestMotifs.test_pssm_caracter_invalido) ... ok
test_pssm_tamanhos_de_sequencia_diferentes (__main__.TestMotifs.test_pssm_tamanhos_de_sequencia_diferentes) ... ok
test_pssm_with_custom_bg_freq (__main__.TestMotifs.test_pssm_with_custom_bg_freq) ... ok
test_pwm_tamanhos_diferentes (__main__.TestMotifs.test_pwm_tamanhos_diferentes) ... ok
test_seq_mais_provavel_pssm (__main__.TestMotifs.test_seq_mais_provavel_pssm) ... ok
test_seq_mais_provavel_pwm (__main__.TestMotifs.test_seq_mais_provavel_pwm) ... ok
test_tabela_contagens_caracter_invalido_dna (__main__.TestMotifs.test_tabela_contagens_caracter_invalido_dna) ... ok
test_tabela_contagens_caracter_invalido_rna (__main__.TestMotifs.test_tabela_contagens_caracter_invalido_rna) ... ok

-------------------------------------------------------------

<unittest.runner.TextTestResult run=10 errors=0 failures=0>

# Árvores filogenéticas





Este algoritmo utiliza uma abordagem de clustering hierárquico baseada em alinhamento de sequências para construir uma árvore filogenética. Considera as similaridades entre as sequências fornecidas e organiza-as em uma estrutura hierárquica.

É implementado o algoritmo de Needleman-Wunsch, usado para calcular a similaridade global entre duas sequências. Ele utiliza a matriz BLOSUM62 para pontuar correspondências e penalidades para inserções ou deleções (gaps). O alinhamento é feito de forma dinâmica, preenchendo uma matriz de pontuação e calculando o melhor alinhamento.

Uma matriz de distâncias é construída calculando a pontuação de alinhamento entre todas as combinações de sequências fornecidas. Isso resulta em uma matriz simétrica, onde cada valor representa a similaridade entre duas sequências.

A função arvore retorna o cluster final representando a árvore filogenética. Além disso, a função formata a saída, exibindo tanto a estrutura hierárquica da árvore quanto as sequências utilizadas, nomeadas sequencialmente para fácil identificação.




# Descrição de alto nível


# Entrada:


*   Lista de sequências (lista_seqs): Contém as sequências que serão analisadas (DNA, RNA ou proteínas).

*   Tipo de sequência (seq_type): Indica o tipo das sequências fornecidas (e.g., "DNA", "RNA" ou "protein").







# Validação:



*   Verifica se a lista de sequências está vazia. Caso esteja, lança uma exceção (ValueError).

*   Valida se todas as sequências são compatíveis com o tipo fornecido utilizando a função validar_sequencias.



# Inicialização:



*   Cada sequência é representada por um identificador único (e.g., "Seq1", "Seq2").

*  Uma matriz de distâncias é criada, onde cada célula contém o escore de alinhamento entre duas sequências, calculado com o algoritmo Needleman-Wunsch usando a matriz de substituição BLOSUM62 (ou pontuações genéricas para DNA/RNA).



# Clustering Hierárquico:



*   Busca o par de clusters com a menor distância:
      *   Percorre a matriz de distâncias para encontrar os dois clusters mais próximos.

*   Atualiza os clusters:
      *  Combina os dois clusters selecionados em um único cluster.

      *   Atualiza a matriz de distâncias usando a distância média entre os novos clusters e os antigos.
    

 *   Repete o processo até que reste apenas um cluster:

      *   O último cluster representa a árvore filogenética final.








# Saída:



*   Uma tupla representando a estrutura hierárquica final dos clusters.
*   A sequencia correspondente a cada SEQ X




In [ ]:
import re

def validar_sequencias(lista_seqs, seq_type):
  # Verifica o tipo de sequência
    if seq_type == 'DNA':
        padrao = re.compile("^[ACGT]+$")
    elif seq_type == 'RNA':
        padrao = re.compile("^[ACGU]+$")
    elif seq_type == 'protein':
        padrao = re.compile("^[ACDEFGHIKLMNPQRSTVWY]+$")
    else:
        raise ValueError("Tipo de sequência incompatível com o selecionado.")
 # Valida cada sequência
    for seq in lista_seqs:
        if not padrao.match(seq): # Se a sequência não corresponder ao padrão, retorna False
            return False
    return True  # Retorna True se todas as sequências forem válidas


class Blosum62:
    def __init__(self):
        tabela = """
   A  R  N  D  C  Q  E  G  H  I  L  K  M  F  P  S  T  W  Y  V  B  Z  X  -
A  4 -1 -2 -2  0 -1 -1  0 -2 -1 -1 -1 -1 -2 -1  1  0 -3 -2  0 -2 -1  0 -4
R -1  5  0 -2 -3  1  0 -2  0 -3 -2  2 -1 -3 -2 -1 -1 -3 -2 -3 -1  0 -1 -4
N -2  0  6  1 -3  0  0  0  1 -3 -3  0 -2 -3 -2  1  0 -4 -2 -3  3  0 -1 -4
D -2 -2  1  6 -3  0  2 -1 -1 -3 -4 -1 -3 -3 -1  0 -1 -4 -3 -3  4  1 -1 -4
C  0 -3 -3 -3  9 -3 -4 -3 -3 -1 -1 -3 -1 -2 -3 -1 -1 -2 -2 -1 -3 -3 -2 -4
Q -1  1  0  0 -3  5  2 -2  0 -3 -2  1  0 -3 -1  0 -1 -2 -1 -2  0  3 -1 -4
E -1  0  0  2 -4  2  5 -2  0 -3 -3  1 -2 -3 -1  0 -1 -3 -2 -2  1  4 -1 -4
G  0 -2  0 -1 -3 -2 -2  6 -2 -4 -4 -2 -3 -3 -2  0 -2 -2 -3 -3 -1 -2 -1 -4
H -2  0  1 -1 -3  0  0 -2  8 -3 -3 -1 -2 -1 -2 -1 -2 -2  2 -3  0  0 -1 -4
I -1 -3 -3 -3 -1 -3 -3 -4 -3  4  2 -3  1  0 -3 -2 -1 -3 -1  3 -3 -3 -1 -4
L -1 -2 -3 -4 -1 -2 -3 -4 -3  2  4 -2  2  0 -3 -2 -1 -2 -1  1 -4 -3 -1 -4
K -1  2  0 -1 -3  1  1 -2 -1 -3 -2  5 -1 -3 -1  0 -1 -3 -2 -2  0  1 -1 -4
M -1 -1 -2 -3 -1  0 -2 -3 -2  1  2 -1  5  0 -2 -1 -1 -1 -1  1 -3 -1 -1 -4
F -2 -3 -3 -3 -2 -3 -3 -3 -1  0  0 -3  0  6 -4 -2 -2  1  3 -1 -3 -3 -1 -4
P -1 -2 -2 -1 -3 -1 -1 -2 -2 -3 -3 -1 -2 -4  7 -1 -1 -4 -3 -2 -2 -1 -2 -4
S  1 -1  1  0 -1  0  0  0 -1 -2 -2  0 -1 -2 -1  4  1 -3 -2 -2  0  0  0 -4
T  0 -1  0 -1 -1 -1 -1 -2 -2 -1 -1 -1 -1 -2 -1  1  5 -2 -2  0 -1 -1  0 -4
W -3 -3 -4 -4 -2 -2 -3 -2 -2 -3 -2 -3 -1  1 -4 -3 -2 11  2 -3 -4 -3 -2 -4
Y -2 -2 -2 -3 -2 -1 -2 -3  2 -1 -1 -2 -1  3 -3 -2 -2  2  7 -1 -3 -2 -1 -4
V  0 -3 -3 -3 -1 -2 -2 -3 -3  3  1 -2  1 -1 -2 -2  0 -3 -1  4 -3 -2 -1 -4
B -2 -1  3  4 -3  0  1 -1  0 -3 -4  0 -3 -3 -2  0 -1 -4 -3 -3  4  1 -1 -4
Z -1  0  0  1 -3  3  4 -2  0 -3 -3  1 -1 -3 -1  0 -1 -3 -2 -2  1  4 -1 -4
X  0 -1 -1 -1 -2 -1 -1 -1 -1 -1 -1 -1 -1 -1 -2  0  0 -2 -1 -1 -1 -1 -1 -4
- -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4 -4  1
"""

        tabela = [T.split() for T in tabela.splitlines() if T]  #Separa a tabela em linhas e coluna

        headers, *resto = tabela # Separa os cabeçalhos das linhas de dados

        from collections import defaultdict
        tab = defaultdict(dict)
         # Preenche o dicionário com os valores da tabela BLOSUM62
        for linha in resto:
            aa, *lst = linha
            for header, score in zip(headers, lst):
                tab[aa][header] = int(score)
        self.tab = tab  # Armazena a tabela no atributo 'tab'

    def subst(self, x, y):
      # Retorna a pontuação de substituição entre dois aminoácidos
        return self.tab[x][y]


def needleman_wunsch(s1, s2, g=-8):
   # Calcula o alinhamento global entre duas sequências usando Needleman-Wunsch
    blosum62 = Blosum62()
    # Inicializa a matriz de pontuação
    score = [[0 for _ in range(len(s1) + 1)] for _ in range(len(s2) + 1)]
    # Preenche as primeiras linhas e colunas da matriz com a penalidade de gap
    for i in range(1, len(s1) + 1):
        score[0][i] = score[0][i - 1] + g
    for j in range(1, len(s2) + 1):
        score[j][0] = score[j - 1][0] + g

    # Preenche o resto da matriz calculando o valor de substituição e penalidade de gap
    for j in range(1, len(s2) + 1):
        for i in range(1, len(s1) + 1):
            match = score[j - 1][i - 1] + blosum62.subst(s1[i - 1], s2[j - 1])
            delete = score[j - 1][i] + g
            insert = score[j][i - 1] + g
            score[j][i] = max(match, delete, insert) # Escolhe a melhor opção (match, delete ou insert)

    return score[-1][-1] # Retorna o valor de alinhamento final


def atualiza_matriz_dist(matriz, clusters, idx1, idx2):
  # Atualiza a matriz de distâncias após combinar dois clusters
    novos_clusters = []
    nova_matriz = []

    # Combine os clusters idx1 e idx2
    novo_cluster = (clusters[idx1], clusters[idx2])
    novos_clusters = [novo_cluster if i == idx1 else clusters[i] for i in range(len(clusters)) if i != idx2]

    # Cria uma nova matriz de distâncias
    for i in range(len(novos_clusters)):
        nova_linha = []
        for j in range(len(novos_clusters)):
            if i == j:
                nova_linha.append(0) # A distância de um cluster consigo mesmo é 0
            elif i == len(novos_clusters) - 1 or j == len(novos_clusters) - 1:
               # Calcula a nova distância entre os clusters combinados
                d1 = matriz[min(idx1, idx2)][max(idx1, idx2)]
                d2 = matriz[min(i, idx1)][max(i, idx1)] if i != idx1 else matriz[min(i, idx2)][max(i, idx2)]
                nova_dist = (d1 + d2) / 2 if i != j else 0
                nova_linha.append(nova_dist)
            else:
                nova_linha.append(matriz[min(i, j)][max(i, j)])
        nova_matriz.append(nova_linha)

    return nova_matriz, novos_clusters # Retorna a nova matriz de distâncias e os novos clusters


def arvore(lista_seqs: list[str], seq_type: str = 'protein', mostrar_resultado: bool = True) -> tuple:
    if not lista_seqs:
        raise ValueError("A lista de sequências está vazia.") # Verifica se a lista de sequências não está vazia

    if not validar_sequencias(lista_seqs, seq_type):
        raise ValueError(f"As sequências fornecidas não são compatíveis com o tipo de sequência '{seq_type}'.")  # Valida as sequências fornecidas

    nomes = [f"Seq{i+1}" for i in range(len(lista_seqs))]  # Gera os nomes das sequências
    matriz = [[0 if i == j else -needleman_wunsch(lista_seqs[i], lista_seqs[j]) for j in range(len(lista_seqs))] for i in range(len(lista_seqs))]

    clusters = nomes[:]  # Inicializa os clusters com os nomes das sequências

    while len(clusters) > 1:  # Enquanto houver mais de um cluster
        min_dist = float('inf')
        i, j = -1, -1
        for x in range(len(matriz)):
            for y in range(x + 1, len(matriz)):
                if matriz[x][y] < min_dist:
                    min_dist = matriz[x][y]
                    i, j = x, y

        # Chama a função atualiza_matriz_dist para atualizar a matriz de distâncias após combinar os clusters
        nova_matriz, clusters = atualiza_matriz_dist(matriz, clusters, i, j)
        matriz = nova_matriz  # Atualiza a matriz de distâncias

    resultado_final = tuple(clusters)

    if mostrar_resultado:
        print("Resultado da Árvore Filogenética:")
        print(f"{(resultado_final)}\n")
        print("Sequências:")
        for nome, seq in zip(nomes, lista_seqs):
            print(f"{nome}: {seq}")

    return resultado_final
# Exemplo
lista_seqs = [
    "ACDEFGHIKLMNPQRSTVWY",
    "DEFGHIKLMNPQRSTVWYA",
    "FGHIKLMNPQRSTVWYAC",
    "HIKLMNPQRSTVWYACDE",
    "IKLMNPQRSTVWYACDEF",
    "KLMNPQRSTVWYACDEFG",
    "LMNPQRSTVWYACDEFGH",
    "MNPQRSTVWYACDEFGHI",
    "NPQRSTVWYACDEFGHIK",
    "PQRSTVWYACDEFGHIKL"
]

seq_type = 'protein'

# Chama a função para gerar a árvore filogenética
arvore(lista_seqs, seq_type);

Resultado da Árvore Filogenética:
(((('Seq1', 'Seq2'), 'Seq3'), ('Seq4', ('Seq5', ('Seq6', ((('Seq7', 'Seq8'), 'Seq9'), 'Seq10'))))),)

Sequências:
Seq1: ACDEFGHIKLMNPQRSTVWY
Seq2: DEFGHIKLMNPQRSTVWYA
Seq3: FGHIKLMNPQRSTVWYAC
Seq4: HIKLMNPQRSTVWYACDE
Seq5: IKLMNPQRSTVWYACDEF
Seq6: KLMNPQRSTVWYACDEFG
Seq7: LMNPQRSTVWYACDEFGH
Seq8: MNPQRSTVWYACDEFGHI
Seq9: NPQRSTVWYACDEFGHIK
Seq10: PQRSTVWYACDEFGHIKL


# Testes Unitários

In [ ]:
import unittest

class TestFilogenia(unittest.TestCase):
    def test_validar_sequencias_DNA(self):
        self.assertTrue(validar_sequencias(["ATCG", "CGTA"], "DNA"))
        self.assertFalse(validar_sequencias(["ATCG", "CGTX"], "DNA"))

    def test_validar_sequencias_RNA(self):
        self.assertTrue(validar_sequencias(["AUCG", "CUGA"], "RNA"))
        self.assertFalse(validar_sequencias(["AUCG", "CXTG"], "RNA"))

    def test_validar_sequencias_protein(self):
        self.assertTrue(validar_sequencias(["ACDE", "FGHI"], "protein"))
        self.assertFalse(validar_sequencias(["ACDE", "FGXI"], "protein"))

    def test_needleman_wunsch(self):
        score = needleman_wunsch("ACG", "AGC")
        self.assertEqual(score, -2)

    def test_atualiza_matriz_dist(self):
        matriz = [[0, 2, 4], [2, 0, 3], [4, 3, 0]]
        clusters = ["Seq1", "Seq2", "Seq3"]
        nova_matriz, novos_clusters = atualiza_matriz_dist(matriz, clusters, 0, 1)
        esperado_matriz = [[0, 2.0], [2.0, 0]]
        esperado_clusters = [("Seq1", "Seq2"), "Seq3"]
        self.assertEqual(nova_matriz, esperado_matriz)
        self.assertEqual(novos_clusters, esperado_clusters)

    def test_arvore_dna_valido(self):
        lista_seqs = ["ACGT", "ACGA", "AGCT", "AGCA"]
        esperado = (((('Seq1', 'Seq2'), 'Seq3'), 'Seq4'),)

        resultado = arvore(lista_seqs, seq_type="DNA", mostrar_resultado=False)
        self.assertEqual(resultado, esperado)


    def test_validar_sequencias_invalidas(self):
        with self.assertRaises(ValueError):
          validar_sequencias(["ATCG", "CGTA"], "XYZ")

    def test_arvore_com_sequencias_vazias(self):
        lista_seqs = []
        with self.assertRaises(ValueError):
            arvore(lista_seqs, "DNA")

    def test_needleman_wunsch_alinhamento_identico(self):
        score = needleman_wunsch("ACGT", "ACGT")
        self.assertGreater(score, 0)

    def test_needleman_wunsch_com_gaps(self):
        score = needleman_wunsch("ACG", "A-G")
        self.assertLess(score, needleman_wunsch("ACG", "ACG"))

    def test_arvore_com_sequencias_vazias(self):
        with self.assertRaises(ValueError) as context:
            arvore([], "DNA")
        self.assertEqual(str(context.exception), "A lista de sequências está vazia.")


unittest.main(argv=[''], exit=False)



...............
----------------------------------------------------------------------
Ran 15 tests in 0.044s

OK
